In [8]:
import os
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import gzip
import pickle as pkl

def loadall_results2(path, n_folds):
    regret = []
    history_actions = []
    history_outcomes = []
    perf = []
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds+1):
            try:
                data = pkl.load(f)
            except EOFError:
                break

            if type(data) == dict:
                regret.append( data['regret'] ) 
                history_actions.append( data['action_history'] )
                history_outcomes.append( data['outcome_history'] ) 
                perf.append( data['pred'] ) 

    return regret, history_actions, history_outcomes, perf

In [25]:
n_folds = 25
horizon = 9999
context = 'MNISTbinary'
model = 'MLP'
case = 'case1'
agent_name = 'EEneuralcbpside_v6' #ineural6, neuronal6

direct = './results/'
path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case,model,context,horizon,n_folds,agent_name) )
regret, action_history,outcome_history, perf = loadall_results2(path, n_folds)


In [44]:
# Process the list to create a format suitable for DataFrame


# Create DataFrame
df = pd.DataFrame(processed_data)


In [29]:
def format_perf(perf, model, dataset, l_label):

    processed_data = []
    i = 0
    for item in perf:
        processed_item = {}
        key_mapping = dict( zip( item.keys(), [10, 50, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 9000]))
        new_dict = {key_mapping[old_key]: value for old_key, value in item.items()}
        for key in new_dict:
            # processed_item[f'accuracy_{key}'] = new_dict[key]['accuracy']
            processed_item[f'f1_{key}'] = new_dict[key]['f1']
        processed_item['Seed'] = i
        processed_item['Model'] = model
        processed_item['Dataset'] = dataset
        processed_item['Approach'] = l_label
        i = i+1
        processed_data.append(processed_item)
    return processed_data



n_folds = 26
horizon = 9999
model = 'MLP'

material = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    'EEneuralcbpside_v6': {'color': [255, 0, 0], 'label': 'Neural-CBP'},  # Red
    'ineural6': {'color': [51, 255, 255], 'label':'IneurAL (official)'},                    # Yellow
    'ineural3': {'color': [0, 0, 255], 'label':'IneurAL (tuned)'},                    # Cyan
    'neuronal6': {'color': [255, 0, 255], 'label':'Neuronal (official)'},                  # Magenta
    'neuronal3': {'color': [160, 160, 160], 'label':'Neuronal (tuned)'},                   # Orange
    'margin': {'color': [160, 160, 160], 'label':'Margin'},
    'cesa': {'color': [0, 0, 255], 'label':'Cesa'},
}

fig = go.Figure( )

data_models = {}
perfs_final = {}
for model in ['MLP']:  # 'LeNet'

    if model == 'MLP':
        n_folds = 25
        datasets = ['MNISTbinary', 'MagicTelescope', 'adult', ] # 'MNIST', 'FASHION', 'covertype', 'shuttle',
    # else:
    #     datasets = ['MNIST', 'FASHION', 'CIFAR10',]
    #     n_folds = 25

    data_regrets = {} 
    data_perfs = {} 
    for data in datasets: 

        # if data in ['MNIST', 'FASHION', 'CIFAR10']:
        #     case = 'case2' 
        # elif data in ['covertype', 'shuttle']:
        #     case = 'game_case_seven'
        # else:
        case = 'case1'
        
        final_regrets = {}
        final_perfs = {}
        for agent_name in material.keys():

            color, l_label = material[agent_name]['color'], material[agent_name]['label']

            r,g,b = color

            # try:
            direct = './results/'
            path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case, model, data, horizon,n_folds,agent_name) )
            print(path)
            regret, action_history,outcome_history, perf = loadall_results2(path, n_folds)
            regret = np.array(regret)
            regret = regret#[1:]
            # regret = regret.astype(np.float32)
            # print(regret.shape)
            
            final_regrets[l_label] = regret[:,-1] 
            final_perfs[l_label] = pd.DataFrame( format_perf(perf, model, data, l_label) )
            # except:
            #     print('hey')
            #     regret = np.zeros((n_folds,horizon))

        if data == 'MNISTbinary':
            data = 'MNISTbinary'
        if data == 'FASHION':
            data = 'Fashion'
            
        data_regrets[data] = final_regrets

        dataframes_list = list(final_perfs.values())
        concatenated_df = pd.concat(dataframes_list, axis=0)
        data_perfs[data] = concatenated_df
        
    data_models[model] = data_regrets
    dataframes_list = list(data_perfs.values())
    concatenated_df = pd.concat(dataframes_list, axis=0)

# fig.show()
# fig.write_image("./figures/case1_{}_{}.pdf".format(model, context) )

./results/case1_MLP_MNISTbinary_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_ineural6.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_ineural3.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_neuronal6.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_neuronal3.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_margin.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_cesa.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_ineural6.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_ineural3.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_neuronal6.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_neuronal3.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_margin.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_cesa.pkl.gz
./results/case1_MLP_adult_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case1_MLP_adult_9999_25_ineural6.pkl.gz
./results/case1_MLP_adult_9999_25_ineural3.pkl.gz
./results/case1_MLP_adu

In [23]:
# Create a mapping from old keys to new keys


new_dict

{10: {'accuracy': 0.7618397707110687, 'f1': 0.658856549710928, 'nverifs': 2},
 50: {'accuracy': 0.7618397707110687, 'f1': 0.658856549710928, 'nverifs': 7},
 100: {'accuracy': 0.7618397707110687, 'f1': 0.658856549710928, 'nverifs': 13},
 250: {'accuracy': 0.7618397707110687, 'f1': 0.658856549710928, 'nverifs': 18},
 500: {'accuracy': 0.7618397707110687, 'f1': 0.658856549710928, 'nverifs': 27},
 750: {'accuracy': 0.7618397707110687, 'f1': 0.658856549710928, 'nverifs': 37},
 1000: {'accuracy': 0.7618397707110687,
  'f1': 0.658856549710928,
  'nverifs': 44},
 2500: {'accuracy': 0.7618397707110687,
  'f1': 0.658856549710928,
  'nverifs': 76},
 5000: {'accuracy': 0.7618397707110687,
  'f1': 0.658856549710928,
  'nverifs': 124},
 7500: {'accuracy': 0.7618397707110687,
  'f1': 0.658856549710928,
  'nverifs': 158},
 9000: {'accuracy': 0.7618397707110687,
  'f1': 0.658856549710928,
  'nverifs': 177}}

In [15]:
final_perfs['Neural-CBP']

,accuracy_1,f1_1,accuracy_51,f1_51,accuracy_88,f1_88,accuracy_137,f1_137,Seed,Model,...,accuracy_94,f1_94,accuracy_50,f1_50,accuracy_96,f1_96,accuracy_227,f1_227,accuracy_281,f1_281
0,0.767436,0.666454,0.781766,0.703452,0.795278,0.734284,0.797598,0.740024,0,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.760338,0.656822,NaN,NaN,NaN,NaN,NaN,NaN,1,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.765934,0.664413,NaN,NaN,NaN,NaN,NaN,NaN,2,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.755425,0.650175,NaN,NaN,NaN,NaN,NaN,NaN,4,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,0.786952,0.733573,NaN,NaN,NaN,NaN,5,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.755835,0.650729,0.785997,0.751336,NaN,NaN,NaN,NaN,6,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.760202,0.656637,NaN,NaN,NaN,NaN,NaN,NaN,7,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.756790,0.652020,NaN,NaN,NaN,NaN,NaN,NaN,9,MLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
import pandas as pd
# pd.DataFrame(data_regrets)
# data_regrets

samples = []
# Iterate over the dictionary to extract data
for model, dataset in data_models.items():
    for data, approach in dataset.items():
        for appr, values in approach.items():
            i = 0
            for val in values:
                # print(i, appr,val)
                samples.append([i, model, data, appr, val])
                i = i+1

# Create a DataFrame
df = pd.DataFrame(samples, columns=['Seed','Model', 'Dataset', 'Approach', 'Value'])
df = pd.merge(df, concatenated_df, on=['Model', 'Dataset', 'Approach', 'Seed'])

# Calculate mean and standard error for each group
# grouped = df.groupby(['Model', 'Dataset', 'Approach'])
# mean = grouped['Value'].mean().reset_index()
# std_error = grouped['Value'].std() / np.sqrt(grouped['Value'].count())
# std_error = std_error.reset_index()
# # Merge the mean and standard error into a single DataFrame
# result = pd.merge(mean, std_error, on=['Model', 'Dataset', 'Approach'])
# result.rename(columns={'Value_x': 'Mean', 'Value_y': 'StdError'}, inplace=True)
# result['merge'] = result['Model'] + result['Approach']
# result['merge2'] = result['Model'] + result['Dataset']

In [31]:
df

,Seed,Model,Dataset,Approach,Value,f1_10,f1_50,f1_100,f1_250,f1_500,f1_750,f1_1000,f1_2500,f1_5000,f1_7500,f1_9000
0,0,MLP,MNISTbinary,Neural-CBP,1353.0,0.606952,0.658141,0.687527,0.655565,0.683871,0.842492,0.860063,0.904477,0.940901,0.950569,0.957999
1,1,MLP,MNISTbinary,Neural-CBP,1352.0,0.626424,0.697114,0.732544,0.668211,0.848383,0.808356,0.869250,0.808366,0.925002,0.928905,0.928030
2,2,MLP,MNISTbinary,Neural-CBP,1556.0,0.348610,0.711514,0.717050,0.780392,0.581689,0.800759,0.752274,0.845571,0.916238,0.944848,0.956199
3,3,MLP,MNISTbinary,Neural-CBP,1298.0,0.386775,0.702732,0.748480,0.752430,0.852819,0.863739,0.862177,0.894895,0.937969,0.952063,0.956900
4,4,MLP,MNISTbinary,Neural-CBP,1346.0,0.331491,0.685806,0.729398,0.622473,0.711874,0.844268,0.875452,0.878335,0.937975,0.953693,0.958202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,20,MLP,adult,Cesa,2518.0,0.090862,0.090862,0.090862,0.090862,0.090862,0.090862,0.090862,0.090862,0.090862,0.090862,0.090862
521,21,MLP,adult,Cesa,2400.0,0.664228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,22,MLP,adult,Cesa,7543.0,0.093335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
523,23,MLP,adult,Cesa,2335.0,0.651098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
df.to_csv('experiment.csv', index=False)

In [32]:
import pandas as pd

# Load the dataframe from the provided file
# df = pd.read_csv('/mnt/data/experiment.csv')

# Columns to exclude from the aggregation
exclude_columns = ['Seed', 'Model', 'Dataset', 'Approach']

# Selecting the columns for which to calculate mean and std
agg_columns = [col for col in df.columns if col not in exclude_columns]

# Grouping by 'Model', 'Dataset', 'Approach' and calculating mean and std
grouped_df = df.groupby(['Model', 'Dataset', 'Approach'])[agg_columns].agg(['mean']) #, 'std'

grouped_df#.head()  # Display the first few rows of the result


Value     f1_10     f1_50  \
                                             mean      mean      mean   
Model Dataset        Approach                                           
MLP   MNISTbinary    Cesa                 4892.52  0.325144  0.325144   
                     IneurAL (official)   3929.48  0.612091  0.752062   
                     IneurAL (tuned)      1726.96  0.456605  0.617890   
                     Margin               3216.72  0.325144  0.325144   
                     Neural-CBP           1351.92  0.418724  0.633312   
                     Neuronal (official)  1721.88  0.488159  0.748138   
                     Neuronal (tuned)     1721.44  0.466616  0.660361   
      MagicTelescope Cesa                 3451.64  0.390923  0.385641   
                     IneurAL (official)   8999.76  0.550029  0.632660   
                     IneurAL (tuned)      6184.00  0.416895  0.520491   
                     Margin               3445.80  0.356284  0.369053   
                     Neural-CBP           3362.60  0.505780  0.536101   
                     Neuronal (official)  4307.36  0.464284  0.637021   
                     Neuronal (tuned)     4296.48  0.400628  0.572652   
      adult          Cesa                 2640.40  0.431680  0.389566   
                     IneurAL (official)   8974.72  0.576245  0.641535   
                     IneurAL (tuned)      4904.08  0.498365  0.534224   
                     Margin               2331.44  0.408914  0.532065   
                     Neural-CBP           2642.76  0.657573  0.621437   
                     Neuronal (official)  4730.92  0.612930  0.665917   
                     Neuronal (tuned)     3717.32  0.501466  0.666578   

                                            f1_100    f1_250    f1_500  \
                                              mean      mean      mean   
Model Dataset        Approach                                            
MLP   MNISTbinary    Cesa                 0.325144       NaN       NaN   
                     IneurAL (official)   0.794428  0.857583  0.884114   
                     IneurAL (tuned)      0.729011  0.811667  0.858388   
                     Margin               0.325144  0.325144  0.325144   
                     Neural-CBP           0.695250  0.732279  0.783948   
                     Neuronal (official)  0.762319  0.804093  0.850893   
                     Neuronal (tuned)     0.686202  0.735821  0.819473   
      MagicTelescope Cesa                 0.385641  0.389531  0.361979   
                     IneurAL (official)   0.655699  0.684788  0.724059   
                     IneurAL (tuned)      0.588354  0.628474  0.708319   
                     Margin               0.369053  0.369053  0.369053   
                     Neural-CBP           0.606089  0.649602  0.673629   
                     Neuronal (official)  0.635156  0.644367  0.668132   
                     Neuronal (tuned)     0.619743  0.689549  0.697437   
      adult          Cesa                 0.339744  0.295502  0.295502   
                     IneurAL (official)   0.602028  0.673637  0.714411   
                     IneurAL (tuned)      0.662131  0.724369  0.692193   
                     Margin               0.587180  0.563870  0.563870   
                     Neural-CBP           0.604684  0.690898  0.694309   
                     Neuronal (official)  0.670008  0.674241  0.681350   
                     Neuronal (tuned)     0.676928  0.683907  0.573205   

                                            f1_750   f1_1000   f1_2500  \
                                              mean      mean      mean   
Model Dataset        Approach                                            
MLP   MNISTbinary    Cesa                      NaN       NaN       NaN   
                     IneurAL (official)   0.906983  0.922758  0.943871   
                     IneurAL (tuned)      0.884166  0.902044  0.937170   
                     Margin               0.325144  0.325144  0.325144

In [7]:
df.to_csv('experiment.csv', index=False)

In [23]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
# Assuming df is your DataFrame and it has columns 'Dataset', 'Value', 'Model'

w = 0.1
def remove_substrings(s, substrings):
    for substring in substrings:
        s = s.replace(substring, '')
    return s


# approaches = result['merge'].unique()
approaches = np.array(['LeNetNeural-CBP', 'LeNetNeuronal (official)', 'LeNetNeuronal (tuned)',
                        'MLPNeural-CBP', 'MLPNeuronal (official)','MLPNeuronal (tuned)', 'MLPIneurAL (official)', 
                        'MLPIneurAL (tuned)',  'MLPCesa', 'MLPMargin' ], dtype=object) 
approaches_names = [ remove_substrings(model, ['LeNet', 'MLP']) for model in approaches ]
default_colors = px.colors.qualitative.Plotly
model_colors = {model_n:col for model_n,col in zip(approaches_names,default_colors) }


# Create an empty figure
fig = go.Figure()


views = []
for appr, appr_name in zip(approaches,approaches_names):

    subset = result[result['merge'] == appr]
    # dec = False
    # if appr_name not in views:
    #     dec = True
    #     views.append(appr_name)
    

    if 'CBP' in appr_name:
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color=model_colors[appr_name],marker_pattern_shape='x',marker_pattern_fgcolor='black', 
            # showlegend = dec,
            # width=w
            ))
    else: 
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color=model_colors[appr_name], 
            # showlegend = dec,
            # width=w
            ))


# Add vertical black lines between data groups

datasets = result['merge2'].unique()
print(datasets)
for i, dataset in enumerate(datasets[:-1]):
    if i == 2:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black")
    else:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black",  line_dash="dot")


siz = 13

# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.31,
#     y0=1.01,
#     x1=0.99,
#     y1=1.01,
#     line_width=2,
#     label=dict(text="MLP", font=dict(size=siz+2)),
# )


# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.31,
#     y0=1.0125,
#     x1=0.31,
#     y1=0.95,
#     line_width=2,
# )


# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.99,
#     y0=1.0125,
#     x1=0.99,
#     y1=0.95,
#     line_width=2,
# )



# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.01,
#     y0=1.01,
#     x1=0.29,
#     y1=1.01,
#     line_width=2,
#     label=dict(text="LeNet", font=dict(size=siz+2)),
# )


# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.01,
#     y0=1.0125,
#     x1=0.01,
#     y1=0.95,
#     line_width=2,
# )

# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.29,
#     y0=1.0125,
#     x1=0.29,
#     y1=0.95,
#     line_width=2,
# )

# fig.update_traces(width=0.05)

# fig.update_traces(
#     marker_pattern_shape='/',  # This sets a grid-like pattern. Use '+' for a cross pattern.
#     marker_pattern_fgcolor='black',  # Foreground color of the pattern
#     marker_pattern_bgcolor='rgba(0,0,0,0)'  # Transparent background color for the pattern
# )

fig.update_layout(
    width=1000,
    height=220,
    plot_bgcolor='white',  # Sets the plot background color
    paper_bgcolor='white',  # Sets the overall figure background color
    barmode='group',
    margin=dict(l=20, r=1, t=25, b=20),  # Small margins
    xaxis=dict(
        tickmode='array',
        tickvals=result['merge2'].unique(),
        ticktext = [ remove_substrings(model, ['LeNet', 'MLP']) for model in result['merge2'].unique() ],
        tickfont=dict(size=siz-2)  # Increase X-axis tick font size
    ),
    yaxis=dict(
        gridcolor='lightgrey',
        title="Final regret ± standard dev.",
        title_standoff=10,
        title_font=dict(size=siz),
        tickfont=dict(size=siz)  # Increase Y-axis tick font size
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,  # Adjust this value to position the legend
        xanchor="center",
        x=0.5,
        font=dict(size=siz)  # Increase legend font size
    ),
)

fig.show()
# fig.write_image("./results_MLP.pdf" )

['MLPMNIST binary' 'MLPMagicTelescope' 'MLPadult']
